In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
import numpy as np

df = pd.read_csv("poker-matches-dataset.csv")
data = pd.read_csv('games.csv')
data_2 = pd.read_csv('games_2.csv')


X = data[['Score']]
y = data['Resultado']

X_2 = data_2[['Score', 'Apostadores', 'Desistentes']]
y_2 = data_2['Resultado']

X_3 = df.drop(columns=['Continuar'])  
y_3 = df['Continuar']      

k = 5

datasets = [
    (X, y),
    (X_2, y_2),
    (X_3, y_3)
]

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

results = []
feature_importances = []

for i, (X, y) in enumerate(datasets):
    print(f"\nTreinando no Dataset {i + 1}")
    scores = cross_val_score(rf_model, X, y, cv=k, scoring='accuracy')
    results.append(scores)

    rf_model.fit(X, y)
    feature_importances.append(rf_model.feature_importances_)

    print(f"Acurácias: {scores}")
    print(f"Média da Acurácia: {np.mean(scores):.4f}")




Treinando no Dataset 1
Acurácias: [0.5 0.6 0.7 0.5 0.9]
Média da Acurácia: 0.6400

Treinando no Dataset 2
Acurácias: [0.8 0.8 0.7 0.7 0.6]
Média da Acurácia: 0.7200

Treinando no Dataset 3
Acurácias: [0.64285714 0.88095238 0.95238095 0.92857143 0.90243902]
Média da Acurácia: 0.8614


In [2]:
from scipy.stats import friedmanchisquare

# Executar o Teste de Friedman para comparar as acurácias
friedman_stat, friedman_p = friedmanchisquare(*results)
print("\nTeste de Friedman:")
print("Estatística:", friedman_stat)
print("p-valor:", friedman_p)

if friedman_p < 0.05:
    print("Há diferenças estatisticamente significativas entre os datasets.")
else:
    print("Não há diferenças estatisticamente significativas entre os datasets.")



Teste de Friedman:
Estatística: 6.0000000000000036
p-valor: 0.04978706836786384
Há diferenças estatisticamente significativas entre os datasets.


In [15]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import friedmanchisquare, rankdata
import importlib
from Orange.evaluation import scoring
_ = importlib.reload( scoring )

# Simulação de dados para teste (substitua pelo seu array `results`)
# results = np.array([[0.8, 0.85, 0.82], [0.78, 0.81, 0.79], [0.76, 0.80, 0.78]])
results = np.array(results)

# Verifica o formato correto: (n_datasets, n_folds)
assert results.shape[1] > results.shape[0], "Os dados devem ter formato (n_datasets, n_folds)."

# Teste de Friedman
friedman_stat, friedman_p = friedmanchisquare(*results)  # Transpõe para formato adequado
print(f"Friedman Test Statistic: {friedman_stat:.4f}, p-value: {friedman_p:.4f}")

if friedman_p < 0.05:
    print("Diferenças estatisticamente significativas detectadas! Realizando análise pós-hoc.")

    # Cálculo das ranks médias
    ranks = np.array([rankdata(-fold) for fold in results])  # Rank por fold
    mean_ranks = np.mean(ranks, axis=0)

    # Nomes dos datasets
    dataset_names = [f"Dataset {i+1}" for i in range(len(mean_ranks))]

    # Diagrama de Diferença Crítica com Orange
    cd = scoring.compute_CD(mean_ranks, n=len(results[0]), alpha="0.05")  # CD para teste de Nemenyi
    scoring.graph_ranks(mean_ranks, dataset_names, cd=cd, width=6)
    plt.show()
else:
    print("Nenhuma diferença estatisticamente significativa detectada. Não será gerado o diagrama.")


Friedman Test Statistic: 6.0000, p-value: 0.0498
Diferenças estatisticamente significativas detectadas! Realizando análise pós-hoc.


AttributeError: module 'Orange.evaluation.scoring' has no attribute 'compute_CD'

In [11]:
print("\nBase 1")
print(f'\nAcurácias por fold: {k_fold}')
print(f'Acurácia média: {k_fold.mean():.2f}')
print("\nBase 2")
print(f'\nAcurácias por fold: {k_fold_2}')
print(f'Acurácia média: {k_fold_2.mean():.2f}')
print("\nBase 3")
print(f'\nAcurácias por fold: {k_fold_3}')
print(f'Acurácia média: {k_fold_3.mean():.2f}')


Base 1

Acurácias por fold: [0.5 0.6 0.7 0.5 0.9]
Acurácia média: 0.64

Base 2

Acurácias por fold: [0.8 0.8 0.7 0.7 0.6]
Acurácia média: 0.72

Base 3

Acurácias por fold: [0.64285714 0.88095238 0.95238095 0.92857143 0.90243902]
Acurácia média: 0.86


In [12]:

print("\n========= Base 1 =========")
importances = decision_model_3.feature_importances_
for feature, importance in zip(X.columns, importances):
    print(f'{feature}: {importance:.2f}')

print("\n========= Base 2 =========")
importances = decision_model_3.feature_importances_
for feature, importance in zip(X_2.columns, importances):
    print(f'{feature}: {importance:.2f}')

print("\n========= Base 3 =========")
importances = decision_model_3.feature_importances_
for feature, importance in zip(X_3.columns, importances):
    print(f'{feature}: {importance:.2f}')


========= Base 1 =========
Score: 0.15

========= Base 2 =========
Score: 0.15
Apostadores: 0.09
Desistentes: 0.23

========= Base 3 =========
Score: 0.15
Perfil: 0.09
Aposta Necessaria: 0.23
Jackpot: 0.27
Jogadores Restantes: 0.08
Desistentes: 0.18


In [5]:
from joblib import dump

#dump(decision_model_3, "modelos/random_forest_model83%.joblib")
#print("Modelo salvo com sucesso!")